In [4]:
!pip install transformers gradio


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: C:\Users\sujat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
!pip install PyMuPDF


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: C:\Users\sujat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
!pip install gradio


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: C:\Users\sujat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
import gradio as gr

# Initialize a global list to store user responses
responses = []

questions = [
    "Hello! It's great to have you here. I'm here to assist you in creating a personalized customer service bot. Let's start with the name of your organization. \nNOTE: Answer all the upcoming questions each in less than 50 words",
    "Could you please specify if your company primarily provides services or sells products?",
    "Could you please provide the current valuation of your company?",
    "Awesome, we are working great together. Now I will need the list of different locations where your company operates and details of the customer services provided at each location.",
    "What technologies are utilized in your workplace?",
    "Can you share the customer service contact numbers for any questions that the chatbot can't handle with the info provided? (Name: contact number – follow this format)",
    "Could you please outline the procedures for collaborating on your projects?"
]

def chatbot_response(user_input, state):
    responses, current_index = state
    if current_index < len(questions):
        if current_index < len(responses):
            responses[current_index] = user_input
        else:
            responses.append(user_input)
        current_index += 1
        if current_index < len(questions):
            response = questions[current_index]
            return (
                response,
                [responses, current_index],
                gr.update(visible=current_index > 0),  # Show Previous button if not at first question
                gr.update(visible=current_index < len(questions) - 1),  # Show Next button if not at last question
                gr.update(visible=current_index == len(questions) - 1),  # Show Submit Data button if at last question
                gr.update(visible=True),  # Always show Response Textbox
                ""  # Clear user input
            )
        else:
            response = "Thank you for providing all the information! Your customer service bot customization is now complete."
            return (
                response,
                [responses, current_index],
                gr.update(visible=False),  # Hide Previous button
                gr.update(visible=False),  # Hide Next button
                gr.update(visible=True),  # Show Submit Data button
                gr.update(visible=False),  # Hide Response Textbox
                ""
            )
    else:
        response = "Thank you for providing all the information! Your customer service bot customization is now complete."
        return (
            response,
            [responses, current_index],
            gr.update(visible=False),  # Hide Previous button
            gr.update(visible=False),  # Hide Next button
            gr.update(visible=False),  # Hide Submit Data button
            gr.update(visible=False),  # Hide Response Textbox
            ""
        )

def previous_question(state):
    responses, current_index = state
    if current_index > 0:
        current_index -= 1
    response = questions[current_index]
    return (
        response,
        [responses, current_index],
        gr.update(visible=current_index > 0),  # Show Previous button if not at first question
        gr.update(visible=current_index < len(questions) - 1),  # Show Next button if not at last question
        gr.update(visible=current_index == len(questions) - 1),  # Show Submit Data button if at last question
        gr.update(visible=True),  # Always show Response Textbox
        responses[current_index]  # Set the text box to the previous response
    )

def submit_data(state):
    save_responses_to_file(state[0])
    return (
        "The 'Submit Data' button has saved the final text file. Thank you for completing the questionnaire!",
        state,
        gr.update(visible=False),  # Hide Previous button
        gr.update(visible=False),  # Hide Next button
        gr.update(visible=False),  # Hide Submit Data button
        gr.update(visible=False),  # Hide Response Textbox
        ""
    )  # Clear user input

def save_responses_to_file(data):
    with open("responses.txt", "w") as file:
        for i, response in enumerate(data):
            file.write(f"Question {i + 1}: {questions[i]}\n")
            file.write(f"Response: {response}\n\n")

def reset_chatbot():
    return (
        questions[0],
        [[], 0],
        gr.update(visible=False),  # Hide Previous button initially
        gr.update(visible=True),  # Show Next button initially
        gr.update(visible=False),  # Hide Submit Data button initially
        gr.update(visible=True),  # Show Response Textbox initially
        ""
    )

# Create Gradio interface
with gr.Blocks() as demo:
    with gr.Column():  # Column to organize the layout
        chatbot_output = gr.Textbox(lines=4, interactive=False)  # Display the current question or final message
        user_input = gr.Textbox(lines=2, placeholder="Type your response here...", interactive=True)  # User input text box for answers
        next_button = gr.Button("Next")
        prev_button = gr.Button("Previous")
        submit_data_button = gr.Button("Submit Data", visible=False)  # Initially hidden

        state = gr.State([[], 0])

        def update_response(user_input, state):
            response, state, prev_visibility, next_visibility, submit_data_visibility, user_input_visibility, user_input = chatbot_response(user_input, state)
            if state[1] < len(responses):
                user_input = responses[state[1]]
            else:
                user_input = ""
            return response, state, prev_visibility, next_visibility, submit_data_visibility, user_input_visibility, user_input

        def handle_submit(user_input, state):
            response, state, prev_visibility, next_visibility, submit_data_visibility, user_input_visibility, user_input = submit_data(state)
            return response, state, prev_visibility, next_visibility, submit_data_visibility, user_input_visibility, ""

        def show_question_state(user_input, state):
            response, state, prev_visibility, next_visibility, submit_data_visibility, user_input_visibility, user_input = update_response(user_input, state)
            return response, state, prev_visibility, next_visibility, submit_data_visibility, user_input_visibility, user_input

        def show_prev_question(state):
            response, state, prev_visibility, next_visibility, submit_data_visibility, user_input_visibility, user_input = previous_question(state)
            return response, state, prev_visibility, next_visibility, submit_data_visibility, user_input_visibility, user_input

        next_button.click(fn=show_question_state, inputs=[user_input, state], outputs=[chatbot_output, state, prev_button, next_button, submit_data_button, user_input, user_input])
        prev_button.click(fn=show_prev_question, inputs=state, outputs=[chatbot_output, state, prev_button, next_button, submit_data_button, user_input, user_input])
        submit_data_button.click(fn=handle_submit, inputs=[user_input, state], outputs=[chatbot_output, state, prev_button, next_button, submit_data_button, user_input, user_input])

        gr.Button("Restart").click(fn=reset_chatbot, inputs=None, outputs=[chatbot_output, state, prev_button, next_button, submit_data_button, user_input, user_input])

        demo.load(fn=reset_chatbot, inputs=None, outputs=[chatbot_output, state, prev_button, next_button, submit_data_button, user_input, user_input])


# Launch the app
demo.launch()


Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [18]:
import gradio as gr

# Function to read the content of the responses.txt file and count the lines
def read_file():
    try:
        with open("responses.txt", "r") as file:
            content = file.read()
        num_lines = content.count('\n') + 1  # Count lines by counting newline characters and adding 1
        return content, f"Number of lines: {num_lines}"
    except FileNotFoundError:
        return "File not found. Please make sure 'responses.txt' exists.", "Number of lines: 0"

# Create Gradio interface
with gr.Blocks() as demo:
    with gr.Column():
        file_content = gr.Textbox(lines=20, interactive=False, label="Content of responses.txt")  # Display file content
        line_count = gr.Textbox(lines=1, interactive=False, label="Line Count")  # Display number of lines

    # Load the content and line count when the app starts
    demo.load(fn=read_file, inputs=None, outputs=[file_content, line_count])

# Launch the app
demo.launch()


Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


In [19]:
import gradio as gr
import io
from pdfminer.high_level import extract_text

# Function to convert PDF to text
def pdf_to_text(pdf_file):
    try:
        text = extract_text(pdf_file.name)
        num_lines = len(text.splitlines())
        return text, num_lines
    except Exception as e:
        return str(e), 0

# Function to handle file upload and conversion
def handle_file_upload(pdf_file):
    if pdf_file is None:
        return "No file uploaded. Please upload a PDF file.", None, None, False
    
    try:
        # Convert PDF to text and count lines
        text, num_lines = pdf_to_text(pdf_file)

        # Prepare text file name and save the text to a file
        text_file_name = "pdf_text.txt"
        with open(text_file_name, "w", encoding="utf-8") as text_file:
            text_file.write(text)
        
        # Read the text file content
        with open(text_file_name, "r", encoding="utf-8") as text_file:
            text_content = text_file.read()
        
        # Prepare response message
        response = (f"Thank you! Your file '{pdf_file.name}' has been uploaded and converted to text.\n\n"
                    f"No. of lines in your PDF file is: {num_lines}\n\n"
                    f"The text has been saved to '{text_file_name}'.")
        
        return response, text_file_name, text_content, True
    
    except Exception as e:
        return f"Error processing the file: {str(e)}", None, None, False

# Initialize responses list
responses = []

# Create Gradio interface
title = "PDF UPLOAD"
description = "We are nearing the end of our questionnaire. Please upload your license terms and conditions policies and any other related information in a single PDF file here to finalize."

demo = gr.Interface(fn=handle_file_upload, inputs="file", outputs=["text", "text", "text"], title=title, description=description)

# Launch the app
demo.launch()


Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


In [20]:
def append_file(file1_path, file2_path):
    try:
        # Open the second file for reading and the first file for appending
        with open(file2_path, 'r', encoding='utf-8') as file2, open(file1_path, 'a', encoding='utf-8') as file1:
            file1.write("\n")  # Add a newline before appending (optional)
            file1.write(file2.read())
        # Return a success message
        return f"Successfully appended '{file2_path}' to '{file1_path}'."
    except FileNotFoundError as e:
        # Return an error message if the file is not found
        return f"Error: {e}"
    except Exception as e:
        # Return a generic error message for other exceptions
        return f"An unexpected error occurred: {e}"

# Example usage
file1_path = 'responses.txt'  # Path to the first text file
file2_path = 'pdf_text.txt'  # Path to the second text file

# Call the function to append the content and capture the result
result = append_file(file1_path, file2_path)
print(result)  # Print the result message


Successfully appended 'pdf_text.txt' to 'responses.txt'.


In [21]:
def read_file(file_path):
    try:
        # Attempt to read the file with 'utf-8' encoding and replace any problematic characters
        with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
            content = file.read()
        return content
    except FileNotFoundError as e:
        return f"Error: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

# Example usage
print(read_file('responses.txt'))


Question 1: Hello! It's great to have you here. I'm here to assist you in creating a personalized customer service bot. Let's start with the name of your organization. 
NOTE: Answer all the upcoming questions each in less than 50 words
Response: Intel Corporation

Question 2: Could you please specify if your company primarily provides services or sells products?
Response: Intel primarily manufactures semiconductor chips and processors.


Question 3: Could you please provide the current valuation of your company?
Response: Intel is valued at approximately $150 billion.


Question 4: Awesome, we are working great together. Now I will need the list of different locations where your company operates and details of the customer services provided at each location.
Response: Santa Clara: Headquarters, Research; Hillsboro: Manufacturing; Bangalore: Software Development

Question 5: What technologies are utilized in your workplace?
Response: AI, IoT, 5G, Quantum Computing, Advanced Manufacturin